In [1]:
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk.data
import nltk
import glob
import os
import re
import string
import pandas as pd
import gensim
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
import numpy as np
from sklearn import preprocessing
from numpy import dot
from numpy.linalg import norm
from pyemd import emd
from sklearn.metrics.pairwise import cosine_similarity

In [132]:
meeting_data = pd.read_pickle("pickleddf.pkl")

In [56]:
meeting_data.head()

,date,text,vector_average
0,2015-03-26,Mayor pro tem I had a que stion for Mr Mckinn...,"[0.03370697, 0.021581726, 0.024202384, 0.09209..."
1,2015-03-26,There are 20 that are signed up for the next ...,"[0.02433229, 0.03419032, 0.00010202286, 0.1011..."
2,2015-03-26,Okay The other thing is Im going to do everyt...,"[0.041835412, 0.021017635, 0.021236643, 0.1323..."
3,2015-03-26,Thanks Hello My name is Jacquie benastante an...,"[0.012454927, 0.028385554, 0.027422577, 0.0879..."
4,2015-03-26,Good afternoon Three of us are here today and...,"[-0.018927153, 0.027210213, 0.046535354, 0.069..."


In [115]:
meeting_data["vector_average"][0].shape

(300,)

In [100]:
# mock user input
text = "bike"

# clean and tokenize user input
tokenized_input = nltk.word_tokenize(text)

tokenized_input

['bike']

In [114]:
# get vectors for input text
#filename = 'GoogleNews-vectors-negative300.bin'
#emb_model = KeyedVectors.load_word2vec_format(filename, binary=True)
vectorized_input = emb_model[tokenized_input]

vectorized_input[0].shape

(300,)

In [104]:
# vectorized input size needs to be 1, 300
test = meeting_data['vector_average'][0]

In [134]:
vectors_1d = []
for nd_vec in meeting_data.vector_average:
    vectors_1d.append(nd_vec[:,np.newaxis])
    
meeting_data["corrected_vectors"] = vectors_1d

In [107]:
meeting_data.head()

,date,text,vector_average,corrected_vectors
0,2015-03-26,Mayor pro tem I had a que stion for Mr Mckinn...,"[0.03370697, 0.021581726, 0.024202384, 0.09209...","[[0.03370697], [0.021581726], [0.024202384], [..."
1,2015-03-26,There are 20 that are signed up for the next ...,"[0.02433229, 0.03419032, 0.00010202286, 0.1011...","[[0.02433229], [0.03419032], [0.00010202286], ..."
2,2015-03-26,Okay The other thing is Im going to do everyt...,"[0.041835412, 0.021017635, 0.021236643, 0.1323...","[[0.041835412], [0.021017635], [0.021236643], ..."
3,2015-03-26,Thanks Hello My name is Jacquie benastante an...,"[0.012454927, 0.028385554, 0.027422577, 0.0879...","[[0.012454927], [0.028385554], [0.027422577], ..."
4,2015-03-26,Good afternoon Three of us are here today and...,"[-0.018927153, 0.027210213, 0.046535354, 0.069...","[[-0.018927153], [0.027210213], [0.046535354],..."


In [96]:
## Failed reshape attempts - keeping for reference
#numpy.reshape(meeting_data['vector_average'][0], 1, 300)
#meeting_data['vector_average'][0].newaxis()
#array.reshape(-1, 1)
#vectorized_input[0].reshape(-1, 1)
#meeting_data["corrected_vectors"][0].reshape(-1, 1)

#test.transpose().shape
#meeting_data['vector_average'][0].shape

In [146]:
meeting_data.head()

,date,text,vector_average,corrected_vectors
0,2015-03-26,Mayor pro tem I had a que stion for Mr Mckinn...,"[0.03370697, 0.021581726, 0.024202384, 0.09209...","[[0.03370697], [0.021581726], [0.024202384], [..."
1,2015-03-26,There are 20 that are signed up for the next ...,"[0.02433229, 0.03419032, 0.00010202286, 0.1011...","[[0.02433229], [0.03419032], [0.00010202286], ..."
2,2015-03-26,Okay The other thing is Im going to do everyt...,"[0.041835412, 0.021017635, 0.021236643, 0.1323...","[[0.041835412], [0.021017635], [0.021236643], ..."
3,2015-03-26,Thanks Hello My name is Jacquie benastante an...,"[0.012454927, 0.028385554, 0.027422577, 0.0879...","[[0.012454927], [0.028385554], [0.027422577], ..."
4,2015-03-26,Good afternoon Three of us are here today and...,"[-0.018927153, 0.027210213, 0.046535354, 0.069...","[[-0.018927153], [0.027210213], [0.046535354],..."


In [145]:
#cosine_similarity(vectorized_input[0].reshape(-1, 1), meeting_data["corrected_vectors"][0].reshape(-1, 1))
cosine_similarity(vectorized_input.reshape(-1, 1), meeting_data["corrected_vectors"][0].reshape(-1, 1))

array([[-1., -1., -1., ...,  1., -1.,  1.],
       [-1., -1., -1., ...,  1., -1.,  1.],
       [-1., -1., -1., ...,  1., -1.,  1.],
       ...,
       [ 1.,  1.,  1., ..., -1.,  1., -1.],
       [-1., -1., -1., ...,  1., -1.,  1.],
       [-1., -1., -1., ...,  1., -1.,  1.]], dtype=float32)

In [144]:
# calculate cosine similarity
cos_sim_input_data = []
for i in meeting_data['corrected_vectors']:
    cos_sim = cosine_similarity([i].reshape(-1, 1), [vectorized_input[0]].reshape(-1, 1))
    cos_sim_input_data.append(cos_sim)

AttributeError: 'list' object has no attribute 'reshape'

In [ ]:
# add input_veclist to df
meeting_data['inputvectors'] = input_veclist



In [ ]:
# sort dataframe in ascending order by distance
meeting_data.sort_values(by=['distance'])

# return url for top 3 rows
return print(meeting_data['text'].head(3))

In [8]:
def activistsdigest():
    # accept user input
    text = input("Write a topic of interest here: ")

    # clean and tokenize user input
    tokenized_input = nltk.word_tokenize(text)
    
    # get vectors for input text
    filename = 'GoogleNews-vectors-negative300.bin'
    emb_model = KeyedVectors.load_word2vec_format(filename, binary=True)
    vectorized_input = emb_model[tokenized_input]

    # calculate WMD distance
    input_veclist = []
    for i in meeting_data['vector_average']:
        distance = emb_model.wmdistance(i, vectorized_input)
        input_veclist.append(distance)

    # add input_veclist to df
    meeting_data['inputvectors'] = input_veclist

    # sort dataframe in ascending order by distance
    meeting_data.sort_values(by=['distance'])

    # return url for top 3 rows
    return print(meeting_data['text'].head(3))